In [22]:
from PolicyNetwork import PolicyNetwork
policy_model = PolicyNetwork(state_dim = 8, action_dim = 16, hidden_dim = 64)

8


In [35]:
from normalization import Normalization
import torch
import onnx
import numpy as np
# from onnx2torch import convert
path = "/home/work/apb-project/ais-deploy-demo-cache/replenishment_wb/replenishment_ppo/output/cache_rp/wb_v1/20250624v0/repl_policy_model.pth"
model_info = torch.load(path, "cpu", weights_only=True) 
# model_info = onnx.load(path)
# model_info = convert(model_info)
policy_model.load_state_dict(model_info["state_dict"])
policy_model.to("cpu")
# guiyihua
state_norm = Normalization(shape=model_info["state_dim"], config=None)
state_norm.running_ms.mean = np.array(model_info["state_norm_mean"])
state_norm.running_ms.std = np.array(model_info["state_norm_std"])
print(state_norm.running_ms.mean)

[2.43343032 0.84709864 0.58516467 0.04097872 0.01482156 0.00359938
 1.84768138 0.83339478]


In [71]:
import json
import torch

# 读取文件
file_path = "/home/work/apb-project/ais-deploy-demo-cache/replenishment_wb/replenishment_ppo/req_test.text"  # 替换为你的文件路径
with open(file_path, "r") as file:
    data = json.load(file)

# 提取 model_input 中的特征
model_input = data["model_input"]
features = ["yesterday_end_stock", 
"predict_arrive_day_1", 
"predict_arrive_day_2",
"predict_arrive_day_3",
"predict_arrive_day_4",
"predict_arrive_day_5",
"lead_time",
"predict_sales_in_lt"]
target_unique_ids = ["247987815950#2917","247987815950#21539"]
all_ids = [item["unique_id"] for item in data["model_input"]]
# 创建一个列表来存储提取的特征
feature_values = []

for item in model_input:
    if item["unique_id"] in all_ids:
        feature_values.append([item[feature] for feature in features])

# 将特征列表转换为 PyTorch 张量
state_1 = state_norm(feature_values,False)
state_1 = torch.tensor(state, dtype=torch.float32).to("cpu")

# 打印张量
print(state_1)


tensor([[-0.1959, -0.2037, -0.1727,  ..., -0.0162,  0.2717, -0.3667],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162,  2.0551, -0.3667],
        [ 0.0774, -0.2037, -0.1727,  ..., -0.0162,  5.6220, -0.3667],
        ...,
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118,  0.0733],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118, -0.3667],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118,  0.0733]])


/tmp/ipykernel_315205/1445768598.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_1 = torch.tensor(state, dtype=torch.float32).to("cpu")


In [72]:
import json
import torch

# 读取文件
file_path = "/home/work/apb-project/ais-deploy-demo-cache/replenishment_wb/replenishment_ppo/req_test_v1.text"  # 替换为你的文件路径
with open(file_path, "r") as file:
    data = json.load(file)

# 提取 model_input 中的特征
model_input = data["model_input"]
features = ["yesterday_end_stock", 
"predict_arrive_day_1", 
"predict_arrive_day_2",
"predict_arrive_day_3",
"predict_arrive_day_4",
"predict_arrive_day_5",
"lead_time",
"predict_sales_in_lt"]
target_unique_ids = ["247987815950#2917","247987815950#21539"]
all_ids = [item["unique_id"] for item in data["model_input"]]
# 创建一个列表来存储提取的特征
feature_values = []

for item in model_input:
    if item["unique_id"] in all_ids:
        feature_values.append([item[feature] for feature in features])

# 将特征列表转换为 PyTorch 张量
state_2 = state_norm(feature_values,False)
state_2 = torch.tensor(state, dtype=torch.float32).to("cpu")

# 打印张量
print(state_2)


tensor([[-0.1959, -0.2037, -0.1727,  ..., -0.0162,  0.2717, -0.3667],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162,  2.0551, -0.3667],
        [ 0.0774, -0.2037, -0.1727,  ..., -0.0162,  5.6220, -0.3667],
        ...,
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118,  0.0733],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118, -0.3667],
        [-0.3326, -0.2037, -0.1727,  ..., -0.0162, -1.5118,  0.0733]])


/tmp/ipykernel_315205/2142183088.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_2 = torch.tensor(state, dtype=torch.float32).to("cpu")


In [73]:
prob_1 = policy_model(state_1)
print(prob_1)

prob_2 = policy_model(state_2)
print(prob_2)

tensor([[0.0562, 0.0611, 0.0597,  ..., 0.0679, 0.0605, 0.0386],
        [0.0528, 0.0635, 0.0567,  ..., 0.0697, 0.0468, 0.0255],
        [0.0380, 0.0708, 0.0571,  ..., 0.0823, 0.0401, 0.0205],
        ...,
        [0.0583, 0.0443, 0.0597,  ..., 0.0658, 0.0697, 0.0581],
        [0.0663, 0.0530, 0.0611,  ..., 0.0595, 0.0733, 0.0600],
        [0.0583, 0.0443, 0.0597,  ..., 0.0658, 0.0697, 0.0581]],
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0562, 0.0611, 0.0597,  ..., 0.0679, 0.0605, 0.0386],
        [0.0528, 0.0635, 0.0567,  ..., 0.0697, 0.0468, 0.0255],
        [0.0380, 0.0708, 0.0571,  ..., 0.0823, 0.0401, 0.0205],
        ...,
        [0.0583, 0.0443, 0.0597,  ..., 0.0658, 0.0697, 0.0581],
        [0.0663, 0.0530, 0.0611,  ..., 0.0595, 0.0733, 0.0600],
        [0.0583, 0.0443, 0.0597,  ..., 0.0658, 0.0697, 0.0581]],
       grad_fn=<SoftmaxBackward0>)
